In [1]:
cash0 = 100000

using JuMP, Cbc, NamedArrays

days = [1, 2, 3, 4, 5]
stocks = [:am, :f, :m, :g, :ap]

values = NamedArray(
[1809.07 168.26 118.76 1188.04 190.35
1810.54 170.79 119.02 1184.24 192.54
1823.97 173.56 119.46 1193.36 194.93
1824.93 177.06 120.05 1206.19 195.76
1831.13 176.45 119.99 1214.68 196.57 ], (days, stocks), ("Days", "Stocks"))

#m = Model(solver=GSolver(OutputFlag=0,BarHomogeneous=1))
m = Model(solver=CbcSolver())

@variable(m, sell[days, stocks] >= 0,Int)
@variable(m, buy[days, stocks] >= 0,Int)
@variable(m, hold[days,stocks] >= 0,Int)
@variable(m, cash[days] >= 0)


@constraint(m, hold_initial[j in stocks], hold[1, j] == buy[1, j])
@constraint(m, cash[1] == cash0 - sum(values[1, j] * buy[1, j] for j in stocks))
@constraint(m, sell_initial[j in stocks], sell[1, j] == 0)
@constraint(m, sum(values[1, j] * buy[1, j] for j in stocks) <= cash0)


@constraint(m, hold_constraint[i in 2:5, j in stocks], hold[i, j] == hold[i - 1, j] + buy[i, j] - sell[i, j])
@constraint(m, cash_constraint[i in 2:5], 
                cash[i] == cash[i - 1] 
                        + sum(values[i, j] * sell[i, j] for j in stocks)
                        - sum(values[i, j] * buy[i, j] for j in stocks))
@constraint(m, buy_constraint[i in 2:5], sum(buy[i, j] * values[i, j] for j in stocks) <= cash[i])
@constraint(m, sell_constraint[i in 2:5, j in stocks], sell[i, j] <= hold[i - 1, j])
            
@constraint(m, sum(hold[5, j] for j in stocks) == 0)

@objective(m, Max, cash[5])
solve(m)

returnVal = getobjectivevalue(m)

println(returnVal)

105594.70000000001


In [6]:
values

5×5 Named Array{Float64,2}
Days ╲ Stocks │     :am       :f       :m       :g      :ap
──────────────┼────────────────────────────────────────────
1             │ 1809.07   168.26   118.76  1188.04   190.35
2             │ 1810.54   170.79   119.02  1184.24   192.54
3             │ 1823.97   173.56   119.46  1193.36   194.93
4             │ 1824.93   177.06   120.05  1206.19   195.76
5             │ 1831.13   176.45   119.99  1214.68   196.57

In [2]:
getvalue(buy)

buy: 2 dimensions:
[1,:]
  [1,am] = 0.0
  [1, f] = 594.0
  [1, m] = 0.0
  [1, g] = 0.0
  [1,ap] = 0.0
[2,:]
  [2,am] = 0.0
  [2, f] = 0.0
  [2, m] = 0.0
  [2, g] = 0.0
  [2,ap] = 0.0
[3,:]
  [3,am] = 0.0
  [3, f] = 0.0
  [3, m] = 0.0
  [3, g] = 0.0
  [3,ap] = 0.0
[4,:]
  [4,am] = 0.0
  [4, f] = 0.0
  [4, m] = 0.0
  [4, g] = 43.0
  [4,ap] = 3.0
[5,:]
  [5,am] = 0.0
  [5, f] = 0.0
  [5, m] = 0.0
  [5, g] = 0.0
  [5,ap] = 0.0

In [3]:
getvalue(cash)

cash: 1 dimensions:
[1] = 53.55999999999382
[2] = 53.55999999999381
[3] = 53.55999999999381
[4] = 52773.75
[5] = 105594.70000000001

In [4]:
getvalue(sell)

sell: 2 dimensions:
[1,:]
  [1,am] = 0.0
  [1, f] = 0.0
  [1, m] = 0.0
  [1, g] = 0.0
  [1,ap] = 0.0
[2,:]
  [2,am] = 0.0
  [2, f] = 0.0
  [2, m] = 0.0
  [2, g] = 0.0
  [2,ap] = 0.0
[3,:]
  [3,am] = 0.0
  [3, f] = 0.0
  [3, m] = 0.0
  [3, g] = 0.0
  [3,ap] = 0.0
[4,:]
  [4,am] = 0.0
  [4, f] = 594.0
  [4, m] = 0.0
  [4, g] = 0.0
  [4,ap] = 0.0
[5,:]
  [5,am] = 0.0
  [5, f] = 0.0
  [5, m] = 0.0
  [5, g] = 43.0
  [5,ap] = 2.9999999999999996

In [5]:
getvalue(hold)

hold: 2 dimensions:
[1,:]
  [1,am] = 0.0
  [1, f] = 594.0000000000001
  [1, m] = 0.0
  [1, g] = 0.0
  [1,ap] = 0.0
[2,:]
  [2,am] = 0.0
  [2, f] = 594.0
  [2, m] = 0.0
  [2, g] = 0.0
  [2,ap] = 0.0
[3,:]
  [3,am] = 0.0
  [3, f] = 594.0000000000001
  [3, m] = 0.0
  [3, g] = 0.0
  [3,ap] = 0.0
[4,:]
  [4,am] = 0.0
  [4, f] = 0.0
  [4, m] = 0.0
  [4, g] = 43.0
  [4,ap] = 3.0000000000000004
[5,:]
  [5,am] = 0.0
  [5, f] = 0.0
  [5, m] = 0.0
  [5, g] = 0.0
  [5,ap] = 0.0